In [11]:
import time
import json
import pandas as pd
import pymysql
from bs4 import BeautifulSoup
from selenium import webdriver

# --- 1. 네이버 영화 랭킹 크롤링 함수 ---
def scrape_naver_movie_rank():
     # Selenium과 BeautifulSoup을 이용해 네이버 영화 랭킹 정보를 가져오는 함수
    URL = "https://movie.naver.com/movie/sdb/rank/rmovie.naver"
    driver = webdriver.Chrome()
    ranks = []

    print("네이버 영화 랭킹 페이지 크롤링을 시작합니다.")

    try:
        driver.get(URL)
        time.sleep(2)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        movie_list = soup.select('table.list_ranking tbody > tr')

        for movie in movie_list:
            title_tag = movie.select_one('div.tit3 > a')
            rank_tag = movie.select_one('td.ac > img')
            point_tag = movie.select_one('td.point')

            if title_tag and rank_tag and point_tag:
                rank = rank_tag['alt']
                title = title_tag.text.strip()
                point = point_tag.text.strip()

                ranks.append({'rank': rank, 'title': title, 'point': point})

        print("영화 정보 추출 완료")

    except Exception as e:
        print(f"크롤링 중 오류 발생 : {e}")

    finally:
        driver.quit()
        print("브라우저를 종료합니다.")

    return ranks

In [12]:
# --- 2. 데이터베이스 저장 함수 ---
def save_to_mariadb(data):
    """
    크롤링한 데이터를 MariaDB에 저장하는 함수
    """
    # DB 연결 설정 (!!! 본인 환경에 맞게 수정하세요 !!!)
    try:
        db = pymysql.connect(
            host="localhost",
            user="webuser",      # 여기에 DB 사용자 이름
            password="webuser",  # 여기에 DB 비밀번호
            database="webdb",    # 접속할 데이터베이스 이름
            charset='utf8mb4'    # 한글 깨짐 방지를 위해 utf8mb4 설정
        )
        cursor = db.cursor()

        # 테이블 생성
        create_table_query = """
        CREATE TABLE IF NOT EXISTS naver_movie_ranks (
            id INT AUTO_INCREMENT PRIMARY KEY,
            rank VARCHAR(10) NOT NULL,
            title VARCHAR(255) NOT NULL,
            point VARCHAR(10)
        )
        """
        cursor.execute(create_table_query)

        # 데이터 저장
        insert_query = """
        INSERT INTO naver_movie_ranks (rank, title, point)
        VALUES (%s, %s, %s)
        """

        cursor.executemany(insert_query, [(d["rank"], d["title"], d["point"]) for d in data])
        db.commit()
        print(f"{len(data)}개의 영화 데이터가 MariaDB에 성공적으로 저장되었습니다.")

    except Exception as e:
        print(f"데이터베이스 작업 중 오류 발생 : {e}")
        if 'db' in locals() and db.open:
            db.rollback()

    finally:
        if 'db' in locals() and db.open:
            cursor.close()
            db.close()

In [13]:
 # --- 3. JSON 파일 저장 함수 ---
def save_to_json(data):
    file_name = "naver_movie_rank.json"

    try:
        with open(file_name, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=4)

        print(f"{len(data)}개의 영화 데이터가 '{file_name}' 파일로 성공적으로 저장되었습니다.")

    except Exception as e:
        print(f"JSON 파일 저장 중 오류 발생 : {e}")

In [14]:
# --- 4. 프로그램의 메인 실행 함수 ---
def main():
    """
    프로그램의 전체 실행 흐름을 제어하는 함수
    """
    print("모든 프로세스를 시작합니다.")

    # 1. 크롤링 함수를 호출하여 영화 데이터를 가져옵니다.
    movie_data = scrape_naver_movie_rank()

    # 2. 가져온 데이터가 있는지 확인합니다.
    if movie_data:
        print(f"\n총 {len(movie_data)}개의 영화 데이터를 성공적으로 가져왔습니다.")
        # 3. DB 저장 함수를 호출하여 가져온 데이터를 DB에 저장합니다.
        save_to_mariadb(movie_data)
    else:
        print("가져온 영화 데이터가 없습니다.")

    print("\n모든 작업 완료")

In [15]:
# --- 5. 스크립트의 시작점 ---
# 이 파일이 직접 실행될 때만 main() 함수를 호출합니다.
if __name__ == '__main__':
    main()

모든 프로세스를 시작합니다.
네이버 영화 랭킹 페이지 크롤링을 시작합니다.
영화 정보 추출 완료
브라우저를 종료합니다.
가져온 영화 데이터가 없습니다.

모든 작업 완료
